In [1]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [10]:
# Verzeichnisse mit den Keypoints & Labels
keypoints_folder = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_gpu"
model_save_path = "/home/haggenmueller/asl_detection/machine_learning/models/lstm/lstm_model.pth"

# Alle `.npy`-Dateien laden
keypoint_files = [f for f in os.listdir(keypoints_folder) if f.endswith(".npy")]

# Label-Extraktion: Unterscheidung zwischen Original & Augmentiertem Format
def extract_label(file_name):
    parts = file_name.split("_")
    if len(parts) >= 3:
        return parts[1]  # Label ist das zweite Element
    return None

# Set aller Labels erstellen
labels = sorted(set(extract_label(f) for f in keypoint_files if extract_label(f) is not None))
num_classes = len(labels)

# Label in Index umwandeln
label_to_index = {label: idx for idx, label in enumerate(labels)}

In [11]:
# Custom Dataset-Klasse
class KeypointDataset(Dataset):
    def __init__(self, keypoints_folder, keypoint_files, label_to_index):
        self.keypoints_folder = keypoints_folder
        self.keypoint_files = keypoint_files
        self.label_to_index = label_to_index

    def __len__(self):
        return len(self.keypoint_files)

    def __getitem__(self, idx):
        file_name = self.keypoint_files[idx]
        file_path = os.path.join(self.keypoints_folder, file_name)

        # Lade Keypoints aus .npy-Datei
        keypoints = np.load(file_path)  # Shape: (max_frames, 99)
        keypoints = torch.tensor(keypoints, dtype=torch.float32)

        # Extrahiere das Label aus dem Dateinamen
        label = extract_label(file_name)
        label_idx = self.label_to_index[label]  # Numerisches Label holen

        return keypoints, torch.tensor(label_idx, dtype=torch.long)  # Sicherstellen, dass Labels long sind

# Dataset & DataLoader erstellen
dataset = KeypointDataset(keypoints_folder, keypoint_files, label_to_index)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

print(f"✅ Dataset geladen: {len(dataset)} Videos mit {len(labels)} Labels.")

✅ Dataset geladen: 6048 Videos mit 168 Labels.


In [12]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=225, hidden_size=256, num_layers=3, output_size=num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        logits = self.fc(lstm_out.mean(dim=1))  # Durchschnitt über alle Frames
        return torch.log_softmax(logits, dim=1)  # Nutze log-Softmax für stabilere Gradienten

print("🔍 Teste vereinfachtes Modell...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

try:
    model = LSTMModel()  # Erst auf CPU erstellen
    print("✅ Modell auf CPU erfolgreich erstellt.")

    model = model.to(device)  # Jetzt auf GPU verschieben
    print("✅ Modell erfolgreich auf GPU geladen.")
except RuntimeError as e:
    print(f"❌ Fehler beim Laden auf die GPU: {e}")

# Verlustfunktion und Optimierer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# Debugging: Prüfen, ob Labels innerhalb des gültigen Bereichs liegen
print(f"Minimales Label: {min(label_to_index.values())}, Maximales Label: {max(label_to_index.values())}")

🔍 Teste vereinfachtes Modell...
✅ Modell auf CPU erfolgreich erstellt.
✅ Modell erfolgreich auf GPU geladen.
Minimales Label: 0, Maximales Label: 167


In [15]:
# Training
num_epochs = 100

# Early Stopping Parameter
early_stopping_patience = 10  # Stoppt, wenn sich der Loss für 10 Epochen nicht verbessert
best_loss = float("inf")
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for keypoints, labels in train_loader:
        keypoints, labels = keypoints.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(keypoints)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f"📌 Epoch {epoch+1}/{num_epochs} - Verlust: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%")

    # Überprüfe Early Stopping Bedingung
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_save_path)  # Bestes Modell speichern
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print(f"🛑 Training gestoppt nach {epoch+1} Epochen, weil keine Verbesserung!")
            break

# Modell speichern
torch.save(model.state_dict(), model_save_path)
print(f"💾 Modell gespeichert unter: {model_save_path}")

📌 Epoch 1/100 - Verlust: 1.3678 - Accuracy: 67.46%
📌 Epoch 2/100 - Verlust: 1.3011 - Accuracy: 69.69%
📌 Epoch 3/100 - Verlust: 1.2437 - Accuracy: 71.20%
📌 Epoch 4/100 - Verlust: 1.2083 - Accuracy: 72.06%
📌 Epoch 5/100 - Verlust: 1.1594 - Accuracy: 72.83%
📌 Epoch 6/100 - Verlust: 1.1618 - Accuracy: 73.07%
📌 Epoch 7/100 - Verlust: 1.0454 - Accuracy: 75.81%
📌 Epoch 8/100 - Verlust: 0.9857 - Accuracy: 77.50%
📌 Epoch 9/100 - Verlust: 0.9751 - Accuracy: 77.46%
📌 Epoch 10/100 - Verlust: 0.9637 - Accuracy: 77.53%
📌 Epoch 11/100 - Verlust: 0.9430 - Accuracy: 77.61%
📌 Epoch 12/100 - Verlust: 0.8681 - Accuracy: 80.13%
📌 Epoch 13/100 - Verlust: 0.7818 - Accuracy: 82.59%
📌 Epoch 14/100 - Verlust: 0.7534 - Accuracy: 83.10%
📌 Epoch 15/100 - Verlust: 0.7586 - Accuracy: 83.02%
📌 Epoch 16/100 - Verlust: 0.7849 - Accuracy: 82.03%
📌 Epoch 17/100 - Verlust: 0.7194 - Accuracy: 83.80%
📌 Epoch 18/100 - Verlust: 0.6531 - Accuracy: 85.15%
📌 Epoch 19/100 - Verlust: 0.5939 - Accuracy: 87.14%
📌 Epoch 20/100 - Verl

In [16]:
# Modell testen
model.eval()
test_sample, test_label = dataset[0]
test_sample = test_sample.unsqueeze(0).to(device)  # Batch-Dimension hinzufügen
output = model(test_sample)
predicted_label = torch.argmax(output, dim=1).item()

print(f"🎯 Testbeispiel: Wahre Klasse = {test_label}, Vorhergesagte Klasse = {predicted_label}")
print("🚀 Training und Test abgeschlossen!")

🎯 Testbeispiel: Wahre Klasse = 98, Vorhergesagte Klasse = 98
🚀 Training und Test abgeschlossen!
